In [1]:
# imports
import os
import sys
import re
import inkml
import glob
import numpy as np
import tensorflow as tf 
from tensorflow.python.framework import ops
import imageio

# Paths to training data

In [2]:
# define paths 
root_path = 'MasterVICO/DataSymbol_Iso/task2-trainSymb2014'

training_Symbols = os.path.join(root_path, 'trainingSymbols')  
training_Junks = os.path.join(root_path, 'trainingJunk')
training_labels_Symbols = 'MasterVICO/DataSymbol_Iso/task2-trainSymb2014/trainingSymbols/iso_GT.txt'
training_labels_Junks = 'MasterVICO/DataSymbol_Iso/task2-trainSymb2014/trainingJunk/junk_GT.txt'
validation_labels_Symbols = 'MasterVICO/DataSymbol_Iso/task2-validation-isolatedTest2013b/iso_GT.txt'
test_labels_Symbols = 'MasterVICO/DataSymbol_Iso/task2-testSymbols2014/testSymbols_2016_iso_GT.txt'


# helper functions definitions

In [3]:
def load_images(folder):
    
    image_size = 28  # Pixel width and height.
    pixel_depth = 255.0  # Number of levels per pixel.

    image_files = os.listdir(folder)
    dataset = np.ndarray(shape=(len(image_files), image_size, image_size), dtype=np.float32)
    print(folder)
    
    num_images = 0
    for image in image_files:
        image_file = os.path.join(folder, image)
        image_data = (imageio.imread(image_file).astype(float) - pixel_depth / 2) / pixel_depth
        dataset[num_images, :, :] = image_data
        num_images = num_images + 1
    dataset = dataset[0:num_images, :, :]
    
    return dataset

In [4]:
def load_labels(file) :  
    Ground_Truth = []
    with open (file) as f:
        lines = f.read().split('\n')[:-1]
        for line in lines : 
            line = line.split(',')
            Ground_Truth.append(line[1])
            
    return np.array(Ground_Truth)

In [5]:
def reformat(dataset): 
    dataset = dataset.reshape((-1, 28, 28, 1)).astype(np.float32)
    return dataset

In [6]:
#training_dataset = load_images('data_png_trainingSymbols') 
#training_junks = load_images('data_png_trainingJunk') 
#validation_dataset = load_images('data_png_task2-validation-isolatedTest2013b')
#test_dataset = load_images('data_png_testSymbols')

In [7]:
#np.save('training_dataset', training_dataset) 
#np.save('training_junks', training_junks) 
#np.save('validation_dataset', validation_dataset) 
#np.save('test_dataset', test_dataset)

# Read labels

In [8]:
#training_Symbols_labels = load_labels(training_labels_Symbols)
#training_junks_labels  = load_labels (training_labels_Junks)
#validation_symbols_labels = load_labels(validation_labels_Symbols)
#test_symbols_labels =  load_labels(test_labels_Symbols)

# save labels 

In [9]:
#np.save('training_Symbols_labels', training_Symbols_labels) 
#np.save('training_junks_labels', training_junks_labels) 
#np.save('validation_symbols_labels', validation_symbols_labels) 
#np.save('test_symbols_labels', test_symbols_labels)

# Load Data

In [10]:
training_dataset = np.load('training_dataset.npy') 
training_junks = np.load('training_junks.npy') 
validation_dataset = np.load('validation_dataset.npy') 
test_dataset = np.load('test_dataset.npy')

In [11]:
print("the shape of training_dataset is ", training_dataset.shape ) 
print("the shape of training_junks is " , training_junks.shape ) 

the shape of training_dataset is  (85802, 28, 28)
the shape of training_junks is  (74284, 28, 28)


In [12]:
train_corpus = np.append(training_dataset,training_junks, axis = 0)

### Load labels 

In [13]:
training_Symbols_labels = np.load('training_Symbols_labels.npy')
training_junks_labels  = np.load ('training_junks_labels.npy')
validation_symbols_labels = np.load('validation_symbols_labels.npy')
test_symbols_labels =  np.load('test_symbols_labels.npy')

# Add Junks_labels to training data symbols_labels

In [14]:
train_labels = np.append(training_Symbols_labels, training_junks_labels)

# keep track of the shapes

In [15]:
print("the shape of the data symbols  is ", train_corpus.shape) 
print("the shape of the data labels is "  , train_labels.shape)

the shape of the data symbols  is  (160086, 28, 28)
the shape of the data labels is  (227623,)


### Reformat data

In [16]:
from sklearn.preprocessing import LabelBinarizer

In [17]:
lb_model = LabelBinarizer() 
lb_model.fit(train_labels)
Y_train = lb_model.transform(train_labels)
Y_validation = lb_model.transform(validation_symbols_labels)
Y_test = lb_model.transform(test_symbols_labels)

X_train = reformat(train_corpus)
X_validation = reformat(validation_dataset) 
X_test = reformat(test_dataset)

# keep track of the shapes 
print ("the shape of  X_train  = {}".format(X_train.shape))
print ("the shape of Y_train  = {}".format(Y_train.shape))
print('\n\n')
print ("the shape of X_validation  = {}".format(Y_validation.shape))
print ("the shape of  X_test = {}".format(Y_test.shape))
print('\n\n')
print ("the shape of X_test  = {}".format( X_test.shape))
print ("the shape of Y_test  = {}".format(Y_test.shape))

the shape of  X_train  =  (160086, 28, 28, 1)
the shape of Y_train  =  (227623, 102)



the shape of X_validation  =  (12504, 102)
the shape of  X_test =  (18435, 102)



the shape of X_test  =  (18435, 28, 28, 1)
the shape of Y_test  =  (18435, 102)


In [18]:
# CNN Neural Network 

In [19]:
## Define place holders

In [36]:
def create_placehoders(batch_size = 1024 , name = 'palce_holders') :
    '''
    a Fnction To define Input and output placeholders
    arg -- 
    return -- 
    '''
    with tf.name_scope(name) : 
    
        X = tf.placeholder(dtype = 'float', shape = [batch_size,28,28,1], name = 'X')
        Y = tf.placeholder(dtype = 'float', shape = [batch_size,102], name = 'Y' ) 
    
    return X,Y                                                

# Architecture 

In [38]:
def initialize_variables(num_hidden = 64, num_labels = 102, patch_size = 5, depth =16, 
                         image_size = 28, name = 'Variables'):
    
    """A function to initialize variables
    args -- 
    returns -- parameters: dict     
    """
    with tf.name_scope(name) : 
    
        W1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, 1, depth],stddev=0.1))
        b1 = tf.Variable(tf.zeros([depth]))
        W2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth],stddev=0.1))
        b2 = tf.Variable(tf.constant(1.0, shape=[depth]))
        W3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
        b3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
        W4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))    
        b4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))


        parameters = {
                      "W1": W1,
                      "b1": b1,
                      "W2": W2,
                      "b2": b2,
                      "W3": W3,
                      "b3": b3,
                      "W4": W4,
                      "b4": b4
                     }
    
    return parameters

## forword Pass

In [39]:
def forword_pass(data, parameters, name = 'forword_pass') :
    
    """define the forword pass, system architecture
    args -- 
    returns --  
    """
    # defines weights and bias
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    W4 = parameters['W4']
    b4 = parameters['b4']

    # define system architecture 
    with tf.name_scope(name) : 
    
        conv = tf.nn.conv2d(data, W1, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + b1)
        conv = tf.nn.conv2d(hidden, W2, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + b2)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, W3) + b3)
        out = tf.add(tf.matmul(hidden, W4), b4)
        
    return out
    

## Backword Pass

In [41]:
def backword_pass(logits, tf_train_labels, name = "compute_loss"):
    
    with tf.name_scope(name): 
    
    #logits = model(tf_train_dataset)
        loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
        
    return loss

In [42]:
def accuracy(predictions, labels):
    
    """
    define accuracy 
    
    """
    
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [43]:
def model (X_train, Y_train, X_validation, Y_validation ,X_test, Y_test, patch_size = 5,
           learning_rate=0.0001 , num_epochs = 1, batch_size = 1024,depth = 16, num_hidden = 64) : 
    
    '''main model function 
    args -- 
    returns -- 
    '''
    #ops.reset_default_graph()
    costs = [] 
    
    
    X, Y = create_placehoders()
    parameters = initialize_variables()
    logits = forword_pass(X, parameters)
    
    # define prediction 
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(forword_pass(X_validation,parameters))
    test_prediction = tf.nn.softmax(forword_pass(X_test, parameters))
    
    # cost
    cost = backword_pass(logits, Y)
    
    with tf.name_scope('train'): 
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)    
    
    init = tf.global_variables_initializer()
    print("parameters initialized ")
    with tf.Session() as sess: 
    
        sess.run(init) 
        
        for epoch in range(num_epochs):
            
            offset = (epoch * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = X_train[offset:(offset + batch_size), :, :, :]
            batch_labels = Y_train[offset:(offset + batch_size), :]
            feed_dict = {X : batch_data, Y : batch_labels}
            costs.append(cost)
            _, l, predictions = sess.run([optimizer, cost, train_prediction], feed_dict=feed_dict)
            if (epoch % 50 == 0):
                
                print('Minibatch loss at epoch %d: %f' % (epoch, l))
                print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                print('Validation accuracy: %.1f%%' % accuracy( valid_prediction.eval(), Y_validation))
                
        print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), Y_test))

In [ ]:
model (X_train, Y_train, X_validation, Y_validation ,X_test, Y_test, patch_size= 5,
           learning_rate = 0.0001, num_epochs = 1 , batch_size = 1024, 
           depth = 16, num_hidden = 64)

parameters initialized 
Minibatch loss at epoch 0: 5.216441
Minibatch accuracy: 0.0%
Validation accuracy: 0.4%
